In [43]:

def recursive_xml_parse(xml_node,global_dict,depth):
    depth = depth+1
    tempdict_inner = xml_node.attrib #extract information from element
    tempdict_inner["tag"+str(depth)] = xml_node.tag #also extract tag and text.
    if (xml_node.text!=None):
        tempdict_inner["text"+str(depth)] = xml_node.text     
    #check if global dictionary has keys that we just extracted
    for key,value in tempdict_inner.items():
        if key in global_dict.keys():
            try: #try to append value if key exists
                global_dict[key].append(value)
            except: #create list if not
                global_dict[key] = [global_dict[key],value]
        else: #add key if it wasnt in global dictionary
            global_dict[key] = value 
    for xml_inner_node in xml_node: #call recursively
        global_dict = recursive_xml_parse(xml_inner_node,tempdict,depth) 
            

    return global_dict

In [77]:
import xml.etree.cElementTree as et
import pandas as pd

name = "VO_RRMSPSV_0000_9965_20200513_0a81f9cc-dc17-4f8e-b30b-f524338ed8bd.xml"
tree=et.parse(name)
root=tree.getroot()
depth = 0
element_list = []
for i in range(1,len(root)):
    tempdict = {}
    tempdict = recursive_xml_parse(root[i],tempdict,depth)
    element_list.append(tempdict)
xml_df = pd.DataFrame(element_list,dtype = "str") 
xml_df.T

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
ИдДок,6e5f9a26-6bb9-40a9-a7ed-15c1fbae6357,6e852d5c-f21c-46a8-885d-f930c2b815d2,7012fabb-dac3-4546-bf1c-684f050e0f77,e15a99bd-16d8-4ee5-851a-fe1e0943a99a,40bb1b25-a53c-49b2-a531-a9cd73a0aa45,01570df6-934f-46d7-a387-10a4e3404dc4,2c403212-cf84-4fbf-a7b0-b47105f6d099,a38b70a8-9b17-46e4-b360-cbb0e7e139d2,a199aa56-d4ed-4094-a340-3bc11ac6c758,a1a56b64-1914-4a9e-8b31-067b8f4c2f40,...,d6a2be09-e76d-4570-9f0f-75a662330bca,b6a00aa5-6cb3-49bd-932f-707924834ebc,80f5a988-31d0-421e-864e-fb15a30ab8f1,3ccbbf94-7685-447f-9ac0-72f0cd092c7a,8bd59978-0723-4fa7-9f54-daec95efb010,eb393256-a5f6-40e9-824d-d3e7ebd165a9,9460a55a-daa4-4b3c-99d9-813424ea6131,6cba3347-3e5c-4a59-a074-737428c07364,a8646064-e703-4b2f-945d-d0b6c790bc7d,df5353d7-bff7-463d-a22b-82370e63dec4
ДатаСост,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,...,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020,10.05.2020
ДатаВклМСП,01.08.2016,01.08.2016,10.03.2018,01.08.2016,10.08.2019,10.12.2019,10.08.2018,10.12.2017,10.04.2019,10.06.2017,...,01.08.2016,10.11.2016,10.02.2019,01.08.2016,10.10.2018,10.08.2018,10.01.2019,01.08.2016,01.08.2016,01.08.2016
ВидСубМСП,2,2,1,1,1,2,2,1,1,1,...,2,1,1,1,2,1,1,2,1,1
КатСубМСП,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ПризНовМСП,2,2,2,2,2,1,2,2,1,2,...,2,2,1,2,2,2,1,2,2,2
СведСоцПред,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
tag1,Документ,Документ,Документ,Документ,Документ,Документ,Документ,Документ,Документ,Документ,...,Документ,Документ,Документ,Документ,Документ,Документ,Документ,Документ,Документ,Документ
ИННФЛ,910901336751,910901942209,NaN,NaN,NaN,910901941639,910901393950,NaN,NaN,NaN,...,910902268984,NaN,NaN,NaN,910901772490,NaN,NaN,910902324117,NaN,NaN
tag2,"[ИПВклМСП, СведМН, СвОКВЭД]","[ИПВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД, СвЛиценз]","[ОргВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ИПВклМСП, СведМН, СвОКВЭД]","[ИПВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]",...,"[ИПВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД, СвЛиценз]","[ОргВклМСП, СведМН, СвОКВЭД]","[ИПВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ИПВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]","[ОргВклМСП, СведМН, СвОКВЭД]"


In [83]:
def condense(row):
    
    templist = []
    if isinstance(row["КодОКВЭД"],list):
        for index in range(len(row["КодОКВЭД"])):
            _dict = {}
            _dict["ИННЮЛ"] = row["ИННЮЛ"]
            _dict["ИННФЛ"] = row["ИННФЛ"]
            _dict["КодОКВЭД"] = row["КодОКВЭД"][index]
            _dict["НаимОКВЭД"] = row["НаимОКВЭД"][index]           
            templist.append(_dict)
    else:
        _dict = {}
        _dict["ИННЮЛ"] = row["ИННЮЛ"]
        _dict["ИННФЛ"] = row["ИННФЛ"]
        _dict["КодОКВЭД"] = row["КодОКВЭД"]
        _dict["НаимОКВЭД"] = row["НаимОКВЭД"]        
        templist.append(_dict)
        
    return templist

In [86]:
series_of_lists = xml_df.apply(condense,axis=1)
import itertools
concated_list = list(itertools.chain.from_iterable(series_of_lists))
condensed_df = pd.DataFrame(concated_list)

In [89]:
condensed_df

,ИННЮЛ,ИННФЛ,КодОКВЭД,НаимОКВЭД
0,NaN,910901336751,47.11,Торговля розничная преимущественно пищевыми пр...
1,NaN,910901336751,47.52,"Торговля розничная скобяными изделиями, лакокр..."
2,NaN,910901336751,47.19,Торговля розничная прочая в неспециализированн...
3,NaN,910901336751,47.21,Торговля розничная фруктами и овощами в специа...
4,NaN,910901336751,47.22,Торговля розничная мясом и мясными продуктами ...
...,...,...,...,...
22716,9109013597,NaN,71.12,"Деятельность в области инженерных изысканий, и..."
22717,9109013597,NaN,71.12.1,"Деятельность, связанная с инженерно-технически..."
22718,9109013597,NaN,71.12.5,Деятельность в области гидрометеорологии и сме...
22719,9109013597,NaN,71.12.6,Деятельность в области технического регулирова...


In [90]:
condensed_df.to_excel('inn_okwed.xlsx')

In [70]:
name = "PrepaidTravelCost.xes_"
tree=et.parse(name)
root=tree.getroot()
depth = 0
element_list = []
for i in range(1,len(root)):
    tempdict = {}
    tempdict = recursive_xml_parse(root[i],tempdict,depth)
    element_list.append(tempdict)
xml_df = pd.DataFrame(element_list,dtype = "str") 
xml_df.T

,0,1,2,3,4,5,6,7,8,9,...,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101
name,Time,Concept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prefix,time,concept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uri,http://www.xes-standard.org/time.xesext,http://www.xes-standard.org/concept.xesext,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tag1,extension,extension,string,trace,trace,trace,trace,trace,trace,trace,...,trace,trace,trace,trace,trace,trace,trace,trace,trace,trace
key,NaN,NaN,concept:name,"[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...",...,"[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or...","[Rfp_id, Permit travel permit number, Task, Or..."
value,NaN,NaN,Prepaid Travel Costs,"[request for payment 73550, UNKNOWN, task 7197...","[request for payment 73552, UNKNOWN, task 7197...","[request for payment 76316, travel permit numb...","[request for payment 73536, UNKNOWN, task 1119...","[request for payment 185372, travel permit num...","[request for payment 76195, travel permit numb...","[request for payment 73737, travel permit numb...",...,"[request for payment 186350, UNKNOWN, task 136...","[request for payment 186361, UNKNOWN, UNKNOWN,...","[request for payment 186366, UNKNOWN, UNKNOWN,...","[request for payment 186371, UNKNOWN, UNKNOWN,...","[request for payment 186355, UNKNOWN, UNKNOWN,...","[request for payment 186623, travel permit num...","[request for payment 13067, travel permit numb...","[request for payment 186629, UNKNOWN, UNKNOWN,...","[request for payment 186617, UNKNOWN, task 152...","[request for payment 186612, UNKNOWN, UNKNOWN,..."
text1,NaN,NaN,NaN,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,...,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t,\n\t\t
tag2,NaN,NaN,NaN,"[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...",...,"[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string...","[string, string, string, string, float, string..."
text2,NaN,NaN,NaN,"[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t]","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...",...,"[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\t\t, \n\t\...","[\n\t\t\t, \n\t\t\t, \n\t\t\

In [71]:
templist = []
xml_df_cut = xml_df[3:]
for x in range(3,len(xml_df_cut)):
    tempdict = {}
    for y in range(len(xml_df_cut["key"])):
        try:
            tempdict[xml_df_cut["key"][x][y]] = xml_df_cut["value"][x][y]
        except:
            pass
    templist.append(tempdict)
df = pd.DataFrame(templist)
df.head().T

,0,1,2,3,4
Rfp_id,request for payment 73550,request for payment 73552,request for payment 76316,request for payment 73536,request for payment 185372
Permit travel permit number,UNKNOWN,UNKNOWN,travel permit number 76315,UNKNOWN,travel permit number 424
Task,task 71977,task 71977,UNKNOWN,task 11190,task 47027
OrganizationalEntity,organizational unit 65463,organizational unit 65463,organizational unit 65463,organizational unit 65463,organizational unit 65457
RequestedAmount,854.5798378437331,854.5798378437331,1173.9577945793626,790.5520731899097,456.43343612267864
Activity,activity 505,activity 505,UNKNOWN,activity 505,activity 505
Permit TaskNumber,UNKNOWN,UNKNOWN,UNKNOWN,task 427,task 427
Permit BudgetNumber,budget 6198,budget 6198,budget 2233,budget 899,budget 425
Permit ProjectNumber,UNKNOWN,UNKNOWN,UNKNOWN,project 2260,project 426
Project,project 503,project 503,project 503,project 503,project 503


In [73]:
df.to_excel("PrepaidTravelCost.xlsx")

In [60]:
name = "test.xhtml"

tree=et.parse(name)
root=tree.getroot()
depth = 0
element_list = []
for i in range(1,len(root)):
    tempdict = {}
    tempdict = recursive_xml_parse(root[i],tempdict,depth)
    element_list.append(tempdict)
xml_df = pd.DataFrame(element_list,dtype = "str") 
xml_df.T

,0
tag1,{http://www.w3.org/1999/xhtml}body
text1,\n \n


In [69]:
name = "test.fb2"

tree=et.parse(name)
root=tree.getroot()
depth = 0
element_list = []
for i in range(1,len(root)):
    tempdict = {}
    tempdict = recursive_xml_parse(root[i],tempdict,depth)
    element_list.append(tempdict)
xml_df = pd.DataFrame(element_list,dtype = "str") 
xml_df

,tag1,tag2,tag3,text3,tag4,text4,{http://www.w3.org/1999/xlink}href,type,tag5,text5,tag6,text6,name,id,content-type,text1
0,{http://www.gribuser.ru/xml/fictionbook/2.0}body,[{http://www.gribuser.ru/xml/fictionbook/2.0}t...,[{http://www.gribuser.ru/xml/fictionbook/2.0}p...,"[Ицхак Адизес, Идеальный руководитель. Почему ...",[{http://www.gribuser.ru/xml/fictionbook/2.0}e...,"[С. Филонович, П. Суворова, Н. Лисицына, В. Му...","[http://www.adizes.com/, http://www.litres.ru/...","[note, note, note, note, note, note, note, not...",[{http://www.gribuser.ru/xml/fictionbook/2.0}e...,"[Москва, сентябрь 2006 г, Для чего нужна эта к...",[{http://www.gribuser.ru/xml/fictionbook/2.0}e...,"[«Кто мудр? Тот, кто учится у всех и каждого. ...",NaN,NaN,NaN,NaN
1,{http://www.gribuser.ru/xml/fictionbook/2.0}body,[{http://www.gribuser.ru/xml/fictionbook/2.0}s...,[{http://www.gribuser.ru/xml/fictionbook/2.0}t...,NaN,[{http://www.gribuser.ru/xml/fictionbook/2.0}p...,"[Сноски, Именно в этом состоит основное отличи...",http://www.adizes.com/,NaN,[{http://www.gribuser.ru/xml/fictionbook/2.0}p...,"[1, 2, Ицхак Адизес, Прим. пер, 3, Ichak Adize...",NaN,NaN,notes,"[n_1, n_2, n_3, n_4, n_5, n_6, n_7, n_8, n_9, ...",NaN,NaN
2,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_001.png,image/png,iVBORw0KGgoAAAANSUhEUgAAA+gAAAGuCAMAAAA049yOAA...
3,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_002.png,image/png,iVBORw0KGgoAAAANSUhEUgAAA+gAAADSCAMAAADudBqAAA...
4,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_003.png,image/png,iVBORw0KGgoAAAANSUhEUgAAA+gAAAEVCAMAAABE7j84AA...
5,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_004.png,image/png,iVBORw0KGgoAAAANSUhEUgAAA+gAAAGbCAMAAABgVEqQAA...
6,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_005.png,image/png,iVBORw0KGgoAAAANSUhEUgAAA+gAAAHICAMAAADqTjDJAA...
7,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_006.png,image/png,iVBORw0KGgoAAAANSUhEUgAAApwAAAMICAMAAABSFcV2AA...
8,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_007.png,image/png,iVBORw0KGgoAAAANSUhEUgAAAvgAAAL4CAMAAAA5/HPRAA...
9,{http://www.gribuser.ru/xml/fictionbook/2.0}bi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i_008.png,image/png,iVBORw0KGgoAAAANSUhEUgAAAoUAAAIdCAMAAAB8wvfIAA...


In [ ]:
text